### 静默度数据统计

In [1]:
from mongo_cli import db,MODE
sid_info = db["sid_info"]
res = sid_info.find({'status':0}).sort("_id",-1).limit(3)
sid_list = [x.get('sid') for x in res]
print sid_list

[u'SID05509b2c38fa89e1f6457b2b1d73fa4d', u'SID2f8dd56188b04f5391a3ea75fe9bc9a1', u'SID6901c3218f77af3d6afeee60a1250720']


In [28]:
res = db['moxie_call_log'].find({}).limit(10)
# res = db['moxie_call_log'].find({}).sort("_id", -1).limit(10)
sid_list = [i.get('sid') for i in res]
print sid_list

[u'SID076bbd3a3fe07ecc43271c3c3906acef', u'SID30ccdbc529ddd7bb154c8c66310d1348', u'SIDafe8ee8a26cf42a96359dfa448d17c6b', u'SID8afab2556cf341c5712f1d288e6dd773', u'SID2b7f21e200f29d5e5ebb12782c623c4e', u'SIDc9cc376c875437ff8334d42249cf1d67', u'SIDb39fb2834f9c490e0d898a67c804e671', u'SID86fc0ed834a8bed3a95335ff6231b787', u'SIDabe912ba00be8a2f339ae4da159d0721', u'SID5d8834c0c9b6685dbaa4ee5fd84360fb']


### 提取数据&加工基础数据

In [30]:
# 提取数据
import json
res = db['moxie_call_log'].find({},{'sid':1,'_id':0}).sort("_id", -1)
sid_list = [i.get('sid') for i in res]
with open('jinmo.sid','w') as fp:
    json.dump(sid_list, fp, indent=4, sort_keys=True)


In [3]:
import json
# 加载待处理sid
with open("jinmo.sid", 'r') as rp:
    sidList = json.loads(rp.read())
print len(sidList)
    
from blank_cul import CulBlankTime
CulBlankTime().deal_blank(sidList[0])

61265


In [7]:
import time
import calendar
import bisect

from dateutil.relativedelta import relativedelta
from dateutil.rrule import rrule, MONTHLY
from datetime import datetime


# def reset_date_start(date_x, month):
#     date_x = date_x + relativedelta(months=month)
#     date_x = "%d-%02d-01 00:00:00"%(date_x.year, date_x.month)
#     return datetime.strptime(date_x, "%Y-%m-%d %H:%M:%S")

# def reset_date_end(date_y, month):
#     date_y = date_y + relativedelta(months=month)
#     date_y_days = calendar.monthrange(date_y.year, date_y.month)[1]
#     date_y = "%d-%02d-%02d 23:59:59"%(date_y.year, date_y.month, date_y_days)
#     return datetime.strptime(date_y, "%Y-%m-%d %H:%M:%S")

BLANK_DEGREE_MAP = {
    'blank_times_count_30': ([0, 1, 2, float('inf')], [0,1,2,3], 0.3), #0.3
    'blank_times_count_90': ([1, 3, 6, float('inf')], [0,1,2,3], 0.25), #0.25
    'blank_times_count_180': ([1, 5, 12, float('inf')], [0,1,2,3], 0.15), #0.15
    'blank_times_count_max': ([120, 240, 480, float('inf')], [0,1,2,3], 0.3) #0.3
}

RESULT_MAPPING = ([0, 1, 2, 3], [0,1,2,3])

import time
# 时间字符串转时间戳（datetimeStr to timestamp）
def dateStrToTimestamp(str_, frmt='%Y-%m-%d %H:%M:%S'):
#     return time.mktime(datetime.strptime(str_, frmt).timetuple())
    return time.mktime(time.strptime(str_, frmt))

def cut_months(term, nums=3, frmt="%Y%m"):
    return [(datetime.strptime(term, frmt) - relativedelta(months=i)).strftime(frmt) for i in xrange(0, nums)]

class CulBlankTime(object):

    def __init__(self):
        self.count_30 = 0
        self.count_90 = 0
        self.count_180 = 0
        self.lenth = 0
        self.max_start = ''
        self.blank_times_list = []
        self.times = []
        self.open_date = ""
        self.last_time = ""
        self.missing_list = []
        
    def _get_data(self, sid):
        data = db['moxie_call_log'].find_one({'sid':sid})
        calls = [dateStrToTimestamp(j.get('time')) for i in data['calls'] for j in i.get('items')]  # 2018-10-31 10:42:05
        smses = [dateStrToTimestamp(j.get('time')) for i in data['smses'] for j in i.get('items')]  # 2018-10-31 10:42:05
        nets = [dateStrToTimestamp(j.get('time')) for i in data['nets'] for j in i.get('items')]  # 2018-10-31 10:42:05
        self.times = calls + smses + nets
        open_date = data.get("open_time", "") # 2015-06-08
        self.open_date = dateStrToTimestamp(open_date, frmt="%Y-%m-%d") if open_date else ""
        self.missing_list =[k for k,v in data.get("month_info").get("month_list").iteritems() if v == -1]
        last_time = data.get("last_modify_time", "")
        self.last_time = dateStrToTimestamp(last_time) if last_time else float(int(time.time()))
#         return times, open_date, missing_list

    def deal_blank(self, sid):
        self._get_data(sid)
        blank_ret = self.times
        open_date = self.open_date
        # 添加最近时间
        blank_ret.append(self.last_time)
        # 处理有开卡日期情况
        if open_date not in ['', u'运营商未透露']:
            query_date = datetime.today() + relativedelta(months=-5)
            query_month = query_date.strftime("%Y%m")
            first_month = query_date.strftime('%Y-%m-01 00:00:00')
            time_first = time.mktime(time.strptime(first_month, "%Y-%m-%d %H:%M:%S"))
            if float(open_date) > time_first:
                time_first = float(open_date)
            else:
                # 处理首月缺失情况，以第一条记录所在月份为第一月
                if query_month in self.missing_list:
                    query_date = datetime.fromtimestamp(blank_ret[0])
                    first_month = query_date.strftime('%Y-%m-01 00:00:00')
                    time_first = time.mktime(time.strptime(first_month, "%Y-%m-%d %H:%M:%S"))
            blank_ret.append(time_first)
#         lens = (len(blank_ret),len(set(blank_ret)))
        reduce(self.count_times, sorted(set(blank_ret)))
        
        # 计算静默度
        cul_data = {
            "blank_times_count_30": str(self.count_30),
            "blank_times_count_90": str(self.count_90),
            "blank_times_count_180": str(self.count_180),
            "blank_times_count_max": "{:.2f}".format(self.lenth),
        }
#         blank_times_degree = self._cul_blank_degree(cul_data)
        return {'sid':sid, 'data':cul_data}

    def _cul_blank_degree(self, data):
        score_total = 0
        for k, v in BLANK_DEGREE_MAP.items():
            idx = bisect.bisect_left(v[0], int(float(data[k])))
            score_total += v[1][idx]*v[2]
        idx_result = bisect.bisect_left(RESULT_MAPPING[0], score_total)
        return RESULT_MAPPING[1][idx_result]
    
    def reset_date_start(self, date_x, flag):
        if flag:
            date_x = date_x + relativedelta(months=1)
            if date_x.strftime("%Y%m") not in self.missing_list:
                return date_x.replace(day=1,hour=0,minute=0,second=0)
            else:
                return self.reset_date_start(date_x, flag)
        else:
            date_x = date_x + relativedelta(months=-1)
            if date_x.strftime("%Y%m") not in self.missing_list:
                date_x = date_x + relativedelta(months=1)
                return date_x.replace(day=1,hour=0,minute=0,second=0)
            else:
                return self.reset_date_start(date_x, flag)
        
    def count_times(self, x, y):
        """
        计数缄默时间
        """
#         lenth_day = int((y-x)/86400)
        # 当月&近3月
        local_mon = datetime.now().strftime("%Y%m")
        recent_3m = cut_months(local_mon)
        date_x = datetime.fromtimestamp(x)
        date_y = datetime.fromtimestamp(y)
        month_x = date_x.strftime("%Y%m")
        month_y = date_y.strftime("%Y%m")
#         print date_x,date_y
        lenth_day = date_y - date_x
        common_list = []
        #处理日期在缺失列表内
        if self.missing_list:
            # 处理边界数据
            if month_x in self.missing_list and month_y in self.missing_list:
                date_x = self.reset_date_start(date_x, 0)
                date_y = self.reset_date_start(date_y, 1)
            elif month_x in self.missing_list:
                date_x = self.reset_date_start(date_x, 1)
            elif month_y in self.missing_list:
                date_y = self.reset_date_start(date_y, 0)
            lenth_day = date_y - date_x
#             print date_x, date_y
            # 组建判定月份
            dates = [dt.strftime("%Y%m") for dt in rrule(freq=MONTHLY, dtstart=date_x, bymonthday=-1).between(date_x, date_y)]
#             print 'dates', dates
            # 如判断月份为当月，list增加当月
            if date_y.month == datetime.now().month and local_mon not in dates:
                dates.append(local_mon)
            # 获取需减去月份
            common_list = list(set.intersection(set(dates),set(self.missing_list)))
#             print 'common_list', common_list
            # 重新生成静默天数
            if local_mon in common_list:
                common_list.remove(local_mon)
                remove_day = date_y - datetime.strptime(local_mon,'%Y%m')
                lenth_day -= remove_day
        blank_day = lenth_day.days
        remain_hours = float(lenth_day.seconds*1.0/3600)
        blank_day -= sum([calendar.monthrange(int(query_date[:4]), int(query_date[-2:]))[1] for query_date in common_list if query_date ])
#         print 'blank_day', blank_day
        if blank_day >= 3:
            lenth_hour = blank_day*24 + remain_hours
            blank_tuple = (str(int(time.mktime(date_x.timetuple()))), str(
                int(time.mktime(date_y.timetuple()))), "{:.2f}".format(lenth_hour))
            self.blank_times_list.insert(0, blank_tuple)
            if x > self.last_time - 2592000:
                self.count_30+=1
            if x > self.last_time - 7776000:
                self.count_90+=1
#             if month_x == local_mon:
#                 self.count_30+=1
#             if month_x in recent_3m:
#                 self.count_90+=1
            self.count_180+=1
            if lenth_hour > self.lenth:
                self.lenth = lenth_hour
                self.max_start = str(int(time.mktime(date_x.timetuple())))
        return y

n = 100
tmp_list = []
for idx, sid in enumerate(sidList[:10000]):
    with open('blank_result_0.json', 'w') as wpt:
        tmp_list.append(CulBlankTime().deal_blank(sid))
        if idx>= n and idx%n==0:
            with open('blank_result_0_tmp.json', 'w') as wp:
                json.dump(tmp_list, wp)
        json.dump(tmp_list, wpt)
        
# from multiprocessing.pool import Pool
# # from multiprocessing.dummy import Pool
# p = Pool(8)
# # parts = [sid_list[i::n] for i in xrange(n)]
# c = CulBlankTime()
# result = p.map(c.deal_blank,sid_list[:10])    
# p.close()
# p.join()
# with open('blank_result_pool.json', 'w') as wpt:
#     json.dump(result, wpt)

In [85]:
l = [i for i in xrange(100)]
n = 8
print [l[i:i + n] for i in xrange(0, len(l), n)]

[l[i::n] for i in xrange(n)]

[[0, 1, 2, 3, 4, 5, 6, 7], [8, 9, 10, 11, 12, 13, 14, 15], [16, 17, 18, 19, 20, 21, 22, 23], [24, 25, 26, 27, 28, 29, 30, 31], [32, 33, 34, 35, 36, 37, 38, 39], [40, 41, 42, 43, 44, 45, 46, 47], [48, 49, 50, 51, 52, 53, 54, 55], [56, 57, 58, 59, 60, 61, 62, 63], [64, 65, 66, 67, 68, 69, 70, 71], [72, 73, 74, 75, 76, 77, 78, 79], [80, 81, 82, 83, 84, 85, 86, 87], [88, 89, 90, 91, 92, 93, 94, 95], [96, 97, 98, 99]]


[[0, 8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88, 96],
 [1, 9, 17, 25, 33, 41, 49, 57, 65, 73, 81, 89, 97],
 [2, 10, 18, 26, 34, 42, 50, 58, 66, 74, 82, 90, 98],
 [3, 11, 19, 27, 35, 43, 51, 59, 67, 75, 83, 91, 99],
 [4, 12, 20, 28, 36, 44, 52, 60, 68, 76, 84, 92],
 [5, 13, 21, 29, 37, 45, 53, 61, 69, 77, 85, 93],
 [6, 14, 22, 30, 38, 46, 54, 62, 70, 78, 86, 94],
 [7, 15, 23, 31, 39, 47, 55, 63, 71, 79, 87, 95]]

### 计算静默度

In [ ]:
BLANK_DEGREE_MAP = {
    'blank_times_count_30': ([0, 1, 2, float('inf')], [0,1,2,3], 0.3), #0.3
    'blank_times_count_90': ([1, 3, 6, float('inf')], [0,1,2,3], 0.25), #0.25
    'blank_times_count_180': ([1, 5, 12, float('inf')], [0,1,2,3], 0.15), #0.15
    'blank_times_count_max': ([5, 10, 20, float('inf')], [0,1,2,3], 0.3) #0.3
}

RESULT_MAPPING = [0, 1, 2, 3]

def cul_blank_degree(data):
        score_total = 0
        for k, v in BLANK_DEGREE_MAP.items():
            idx = bisect.bisect_left(v[0], int(float(data[k])))
            score_total += v[1][idx]*v[2]
        return score_total
    
"""

# 评分规则
RATING_RULES = [
    ([-8887, 1, 2, 7], [-17, 25, 17, -25, -50]),  # hd_maxofflinedaysin3months
    ([1], [23, -53]),  # dhb_callTelTotalNumCollHist
    ([-8887, 30, 55, 118], [-25, -8, 15, 30, 88]),  # dinner_rent_fee
    ([-8887, 15, 25, 40, 52, 72], [-9, -69, -25, -17, -1, 31, 42]),  # innet_months
    ([-8887, 0.01, 0.02, 0.03, 0.05], [-18, 27, 11, -21, -24, -63]),  # hd_contactearlymorningrate
]

def cul(item):
    score_total = 603
    for n,i in enumerate(RATING_RULES):
        idx = bisect.bisect_left(i[0], item[1][n])
        score_total += i[1][idx]
    return score_total

"""
    
# 计算静默度
cul_data = {
    "blank_times_count_30": str(self.count_30),
    "blank_times_count_90": str(self.count_90),
    "blank_times_count_180": str(self.count_180),
    "blank_times_count_max": "{:.2f}".format(self.lenth),
}

blank_times_degree = cul_blank_degree(cul_data)

In [71]:
3%4


3

In [3]:
map(str, [i for i in xrange(10)])

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']